In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [2]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = Chroma(
    collection_name="heritage_collection",
    embedding_function=embedding_model,
    persist_directory="chroma_db"
)

# 검색
query = "조선시대의 목조건축"
results = vector_store.similarity_search(query, k=5)

for r in results:
    print("------")
    print(r.page_content)
    print(r.metadata)


C:\Users\jhwoo\AppData\Local\Temp\ipykernel_10548\1133054664.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


------
설명: ­1904년경 건립된 것으로 보이는 전통목조 주택.가구기법 및 치목기법이 우수하고 공간 및 형태의 구성미와 조형미가 뛰어남
{'소유자(소유단체)': '김＊＊＊', '관리자': '김＊＊＊', 'row': 2221, '소재지(상세)': '경북 봉화군 봉화읍 토일길 156-12 (유곡리)', 'source': 'https://www.heritage.go.kr/heri/cul/culSelectDetail.do?culPageNo=223&region=2&searchCondition=&searchCondition2=&s_kdcd=00&s_ctcd=37&ccbaKdcd=79&ccbaAsno=02180000&ccbaCtcd=37&ccbaCpno=4413702180000&ccbaCndt=&ccbaLcto=00&stCcbaAsno=&endCcbaAsno=&stCcbaAsdt=&endCcbaAsdt=&ccbaPcd1=&chGubun=&header=region&returnUrl=%2fheri%2fcul%2fculSelectRegionList.do&pageNo=1_1_3_1&sngl=Y', '수량/면적': '1동, 1층, 연면적 130.84㎡', '관리자(관리단체)': '김＊＊＊', '명칭': '봉화 유곡리 근대 한옥(奉化 酉谷里 近代 韓屋)', '종목': '국가등록문화유산', '소재지': '경북 봉화군', '시대': '1904년경', '분류': '등록문화유산 / 기타 / 주거숙박시설', '지정(등록)일': '2005.11.11'}
------
설명: 범어사 목조(木槽)는 통나무를 절반으로 자른 후 장방형으로 파낸 내부 밑바닥에 배수를 위한 방형 및 2개의 원형 구멍 배수구 및 바깥 양쪽 끝에는 고리모양의 홈이 뚫려있는 牛馬의 구유와 같은 나무통 형태이다. 목조의 외부 표면에 새겨진 명문으로 보아 영조 33년(1757) 10월에 취정사에서 제작된 후 수레로 가져왔음을 알 수 있고, 조선시대 사찰에서의 종이 부역과 관련된 유물이기도 하여 부산광역시 민속문화유산적 가치

In [27]:
retriever = vector_store.as_retriever(search_kwargs={"k":10})

In [28]:
query = "백제의 문화 특징을 알려줘."

result = retriever.invoke(query)
result

[Document(metadata={'source': 'https://www.heritage.go.kr/heri/cul/culSelectDetail.do?culPageNo=15&region=2&searchCondition=&searchCondition2=&s_kdcd=00&s_ctcd=35&ccbaKdcd=13&ccbaAsno=03470000&ccbaCtcd=35&ccbaCpno=1333503470000&ccbaCndt=&ccbaLcto=00&stCcbaAsno=&endCcbaAsno=&stCcbaAsdt=&endCcbaAsdt=&ccbaPcd1=&chGubun=&header=region&returnUrl=%2fheri%2fcul%2fculSelectRegionList.do&pageNo=1_1_3_1&sngl=Y', '지정(등록)일': '1991.02.26', '소재지': '전북 익산시', '소유자(소유단체)': '익산시 등', '소재지(상세)': '전북 익산시 웅포면 입점리 산174번지 외 3필', '관리자': '익산시', '종목': '사적', '관리자(관리단체)': '익산시', 'row': 143, '수량/면적': '76,056㎡', '시대': '5c중엽', '명칭': '익산 입점리 고분(益山 笠店里 古墳)', '분류': '유적건조물 / 무덤 / 무덤 / 고분군'}, page_content='설명: 전북특별자치도 익산군 입점리 칠목재 구릉 중턱에 있는 백제무덤들이다.\r\n\r\n1986년 이 마을의 한 고등학생이 칡을 캐다가 금동제 모자 등을 발견·신고하여 알게되었다. 긴급발굴을 하여 8기의 무덤을 확인하였는데, 1호를 제외하고는 파손이 심하였다.\r\n\r\n1호 무덤은 널방(현실)과 널길(연도)을 갖춘 굴식돌방무덤(횡혈식석실묘) 이다. 또한 천장은 네 벽을 맞추어 쌓아 올리면서 마지막 4장을 덮은 활천장(궁륭식천장)을 이루었다. 출토된 유물로는 토기류·금동모자와 금귀고리, 유리구슬 등의 장신구류·말갖춤(마구)·철기 등 백제 중요 유물이 수습되었다.\r

In [29]:
template = """
[Instruction]
당신은 한국 문화유산 전문가입니다.
한국 문화유산에 관한 데이터는 대부분 chroma_db에 저장되어 있습니다.
당신에게는 사용자 질문과, chroma_db로부터 획득한 정보가 제공됩니다.
한국 문화유산에 대한 질문이 들어온다면,
chroma_db에 저장되어 있는 데이터를 기반으로 사용자의 질문에 답변해주세요.

[사용자 질문]
{query}

[chroma_db 정보]
{chroma_db_info}
"""

In [30]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import *
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

prompt_template = PromptTemplate(template=template)

model = ChatOpenAI(model_name="gpt-4o-mini")

parser = StrOutputParser()

In [31]:
runn = RunnableParallel(
    {
        "chroma_db_info":retriever,
        "query":RunnablePassthrough()
    }
)

In [32]:
chain = runn|prompt_template|model|parser

In [33]:
from IPython.display import Markdown
query = "백제의 문화유산이 많은 대한민국의 지역은 어디야?"

In [34]:
res = chain.invoke(query)
Markdown(res)

백제의 문화유산이 많은 대한민국의 지역은 전라북도입니다. 특히 익산시와 정읍시에 많은 고분군이 위치하고 있습니다.

1. **익산 입점리 고분군** - 이곳은 백제무덤들이 밀집해 있으며, 5세기경에 만들어진 것으로 추정되는 고분이 포함되어 있습니다. 출토된 유물로는 금동제 모자와 장신구 등이 있어 백제와 일본 간의 문화교류를 짐작하게 합니다. [자세한 정보](https://www.heritage.go.kr/heri/cul/culSelectDetail.do?culPageNo=15&region=2&ccbaAsno=03470000)

2. **정읍 은선리와 도계리 고분군** - 여기에는 275여 기의 백제 고분이 분포하고 있으며, 사적 지정된 56기의 횡혈식 석실분이 포함되어 있습니다. 이 고분군은 백제의 중앙과 지방, 대외 관계에 대한 중요한 자료로 평가받고 있습니다. [자세한 정보](https://www.heritage.go.kr/heri/cul/culSelectDetail.do?culPageNo=16&region=2&ccbaAsno=05430000)

이 외에도 전라북도 김제, 화성시도 백제 문화유산이 발견된 지역입니다. 전라북도는 백제 고분군이 다양하게 분포하고 있어 백제의 문화유산을 심층적으로 이해하는 데 중요한 지역입니다.

In [ ]:
# [지피티가 준 주소]
# https://www.heritage.go.kr/heri/cul/culSelectDetail.do?culPageNo=15&region=2&ccbaAsno=03470000
# [실제 주소]
# https://www.heritage.go.kr/heri/cul/culSelectDetail.do?pageNo=1_1_1_1&sngl=Y&ccbaCpno=1333503470000

# 왜 이럴까?